In [10]:
import pandas as pd
import os
import ast
import numpy as np
import json
from tqdm import tqdm

In [11]:
dir_path = "/aienglish-cfs/datasets/raw_events/prod/"
dfs = []
for file in os.listdir(dir_path):
    if not file.startswith("user"):
        tmp_df = pd.read_parquet(os.path.join(dir_path, file))
        print(f"{file}:{tmp_df.shape}")
        dfs.append(tmp_df)
df = pd.concat(dfs, ignore_index=True)
print(df.columns)
print(df.shape)
df[df["has_llm_info"]==True].iloc[:5].to_csv("test.csv", index=False)

2025-08-07.parquet:(583, 35)
2025-08-08.parquet:(1014, 35)
2025-08-09.parquet:(931, 35)
2025-08-10.parquet:(988, 35)
2025-08-11.parquet:(958, 35)
2025-08-12.parquet:(510, 35)
2025-08-13.parquet:(726, 35)
2025-08-14.parquet:(1219, 35)
2025-08-15.parquet:(882, 35)
2025-08-16.parquet:(768, 35)
2025-08-17.parquet:(462, 35)
2025-08-18.parquet:(737, 35)
2025-08-19.parquet:(666, 35)
2025-08-20.parquet:(473, 35)
2025-08-21.parquet:(1275, 35)
2025-08-22.parquet:(1715, 35)
2025-08-23.parquet:(1257, 35)
2025-08-24.parquet:(1275, 35)
2025-08-25.parquet:(800, 35)
2025-08-26.parquet:(973, 35)
2025-08-27.parquet:(1426, 35)
2025-08-28.parquet:(2014, 35)
Index(['ts', 'student_id', 'log_event_type', 'role', 'lesson_id', 'phase_id',
       'session_id', 'trace_id', 'has_llm_info', 'llm_messages',
       'llm_m_config', 'llm_response', 'game_task_game_name',
       'game_task_difficulty', 'game_task_game_goal',
       'game_task_trigger_hint_count', 'game_task_game_duration_seconds',
       'game_task_tar

/tmp/ipykernel_3698652/3934215801.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dfs, ignore_index=True)


In [12]:
print(f"总记录数: {len(df)}")
print(f"游戏任务数: {len(df[df['log_event_type'] == 'game_task'])}")
print(f"发音任务数: {len(df[df['log_event_type'] == 'pronunciation_task'])}")
print(df["game_task_game_name"].unique())

总记录数: 21652
游戏任务数: 2102
发音任务数: 19550
[None 'PAPER_PLANE_1740587562289' 'ICEBREAKER_ROBOT_WORLD_1740587562288'
 'ALBUM_ROBOT_WORLD_1740587562279' 'PAPER_PLANE_1740576495679'
 'CATCH_BALL_GAME_1743432009736' 'PAPER_PLANE_1743434111080'
 'PAPER_PLANE_1745909257846' 'PAPER_PLANE_1745909449148'
 'ICEBREAKER_ROBOT_WORLD_1741170755397' 'PAPER_PLANE_1740316881184'
 'CATCH_BALL_GAME_1740318510088' 'ALBUM_ROBOT_WORLD_1740478387683'
 'ALBUM_ROBOT_WORLD_1740311986161' 'CATCH_BALL_GAME_1744883285938'
 'ICEBREAKER_ROBOT_WORLD_NEW_1744883806483' 'PAPER_PLANE_1740570864562'
 'CATCH_BALL_GAME_1740570864563' 'ICEBREAKER_ROBOT_WORLD_1740570864562'
 'ALBUM_ROBOT_WORLD_1740570864553' 'ALBUM_ROBOT_WORLD_1740047482230'
 'ICEBREAKER_ROBOT_WORLD_1740576495679' 'ALBUM_ROBOT_WORLD_1740576495668'
 'PAPER_PLANE_1745830730131' 'PAPER_PLANE_1745830753854'
 'PAPER_PLANE_1745830790523' 'CATCH_BALL_GAME_1740586655696'
 'CATCH_BALL_GAME2_1740586655696' 'PAPER_PLANE_1740586655696'
 'ALBUM_ROBOT_WORLD_1740586655686' 'CATC

In [35]:
student_ids = df["student_id"].unique()
print(len(student_ids))
print(f"student_ids: {student_ids}")

204
student_ids: ['5611' '44395' '42476' '20' '43966' '42792' '42596' '43667' '42103'
 '43558' '45066' '45069' '43853' '10' '41994' '30067' '43740' '41835'
 '42853' '45077' '40511' '40855' '41037' '43624' '43705' '42417' '44865'
 '45071' '44881' '42675' '42501' '45065' '44892' '44372' '44373' '44301'
 '40856' '42418' '41763' '42921' '43619' '44649' '45087' '41384' '41219'
 '44451' '44958' '41745' '45086' '44868' '44670' '44300' '44653' '40888'
 '45091' '42523' '45076' '45096' '45094' '42366' '43457' '45097' '45098'
 '41828' '44664' '44276' '41824' '44659' '43870' '42935' '45099' '44855'
 '42959' '44864' '41566' '44667' '45108' '44643' '45014' '45109' '42326'
 '45105' '44278' '44606' '42698' '45124' '43968' '41970' '44354' '45118'
 '44047' '45138' '41593' '45137' '42245' '45043' '45147' '44286' '40448'
 '40166' '43725' '42888' '44880' '43355' '43880' '45162' '42321' '42187'
 '45163' '42055' '45165' '45166' '45072' '41515' '45164' '42949' '42399'
 '45175' '45181' '44437' '45142' '43678' 

In [28]:
def calculate_spell_scores(st_infos):
    scores = []
    for st_info in st_infos:
        if st_info=="None":
            scores.append(0)
        elif st_info=="{}":
            scores.append(0)
        else:
            data = ast.literal_eval(st_info)
            try:
                scores.append(data["result"]["overall"])
            except:
                print(f"error: {st_info}")
    return scores

In [29]:
import re

def calculate_fa_scores(df):
    scores = []
    for _, row in df.iterrows():
        if (row["pronunciation_task_standard_answer"] is None) or (row["pronunciation_task_st_info"]=="{}"):
            scores.append(0)
        else:
            standard_answer = row["pronunciation_task_standard_answer"]
            data = ast.literal_eval(row["pronunciation_task_st_info"])
            score = data["result"]["overall"]
            # print(standard_answer)
            if re.match(r"^(Hi|Hello)(.+)$", standard_answer, re.IGNORECASE):
                # print("level hello")
                # Hello, N.
                scores.append(0.3 * score)
            elif re.match(r"^[A-Z]$", standard_answer, re.IGNORECASE):
                # print("correct letter")
                # N
                scores.append(0.3 * score)
            elif re.match(r"^([a-z], )+[a-z]$", standard_answer, re.IGNORECASE):
                # print("correct letter")
                # e, s, e
                scores.append(0.3 * score)
            elif re.match(r"^[a-z]+$", standard_answer, re.IGNORECASE):
                # print("correct word")
                # newt
                scores.append(0.3 * score)
            elif re.match(r"^(/(.+)/)+$", standard_answer, re.IGNORECASE):
                # print("level 0")
                # /n/
                scores.append(0.3 * score)
            elif re.match(r"^(.+) says /(.+)/\.$", standard_answer, re.IGNORECASE):
                # print("level 1")
                # N says /n/.
                scores.append(0.5 * score)
            elif re.match(r"^([a-z], )+/(.+)/$", standard_answer, re.IGNORECASE):
                # print("level 1")
                # e, s, e, /iːz/
                scores.append(0.5 * score)
            elif re.match(r"^(/(.+)/).+[a-z]+$", standard_answer, re.IGNORECASE):
                # print("level 2")
                # /n/ /n/ newt
                scores.append(0.7 * score)
            else:
                # print("level 3")
                # N says /n/, /n/ /n/ newt.
                scores.append(score)
    return scores

In [30]:
def calculate_game_scores(game_df):
    game_scores = []
    for _, row in game_df.iterrows():
        operation_sequence = row["game_task_operation_sequence"]
        if (operation_sequence is None):
            game_scores.append(0)
            continue
        elif (len(operation_sequence) == 0):
            game_scores.append(0)
            continue
        total_operation_count = 0
        correct_operation_count = 0
        for i in range(len(operation_sequence)):
            total_operation_count += 1
            operation = json.loads(operation_sequence[i])
            if operation["correct"]:
                correct_operation_count += 1
        hint_count = 0 if pd.isna(row["game_task_trigger_hint_count"]) else row["game_task_trigger_hint_count"]
        game_score = 1.0 * (correct_operation_count - hint_count) / total_operation_count
        if row["phase_id"].startswith("ICEBREAKER_ROBOT_WORLD"):
            game_score = 40 * game_score
        elif row["phase_id"].startswith("PAPER_PLANE"):
            game_score = 40 * game_score
        elif row["phase_id"].startswith("ALBUM_ROBOT_WORLD"):
            game_score = 60 * game_score
        elif row["phase_id"].startswith("CATCH_BALL_GAME"):
            game_score = 20 * game_score
        else:
            print("unknown phase id: ", row["phase_id"])
        game_scores.append(game_score)
    return game_scores

In [31]:
import re
def find_city_id(df):
    city_id = None
    for _, row in df.iterrows():
        if row["has_llm_info"]==True:
            message = row["llm_messages"]
            pattern = r"城市所在地：(.*?)(?=\s)"
            city_id = re.search(pattern, message[-1]["content"], re.DOTALL | re.IGNORECASE).group(1).strip()
            if city_id:
                return city_id
    return city_id


In [32]:
column_names = ["student_id", "lesson_id", "city_id", "ts", "spell_score", "fa_score", "game_score", "last_1_spell_score", "last_1_fa_score", "last_1_game_score", "last_2_spell_score", "last_2_fa_score", "last_2_game_score", "last_3_spell_score", "last_3_fa_score", "last_3_game_score"]
feature_df = pd.DataFrame(columns=column_names)

for student_id in tqdm(student_ids):
    lesson_ids = df[df["student_id"] == student_id]["lesson_id"].unique()
    lesson_ids = [int(lesson_id) for lesson_id in lesson_ids]
    for lesson_id in np.sort(lesson_ids):
        # print(f"student_id: {student_id}, lesson_id: {lesson_id}")
        single_lesson_df = df[(df["student_id"] == student_id) & (df["lesson_id"] == str(lesson_id))]
        city_id = find_city_id(single_lesson_df)
        # print(f"total events: {single_lesson_df.shape[0]}")
        single_lesson_df = single_lesson_df.sort_values(by="ts", ascending=True)
        spell_scores = calculate_spell_scores(single_lesson_df[single_lesson_df["pronunciation_task_task_type"] == "拼读"]["pronunciation_task_st_info"].values)
        total_spell_score = 0 if len(spell_scores) == 0 else np.mean(spell_scores)
        fa_scores = calculate_fa_scores(single_lesson_df[single_lesson_df["pronunciation_task_task_type"] == "跟读"])
        total_fa_score = 0 if len(fa_scores) == 0 else np.mean(fa_scores)
        game_scores = calculate_game_scores(single_lesson_df[single_lesson_df["log_event_type"]=="game_task"])
        total_game_score = 0 if len(game_scores) == 0 else np.mean(game_scores)
        # print(f"spell_score: {total_spell_score}, fa_score: {total_fa_score}, game_score: {total_game_score}")
        index = len(feature_df)
        feature_df.loc[index] = [int(student_id), str(lesson_id), city_id, single_lesson_df.iloc[0]["ts"], total_spell_score, total_fa_score, total_game_score, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        single_student_df = feature_df[feature_df["student_id"] == int(student_id)].sort_values(by="lesson_id", ascending=False)
        if single_student_df.shape[0] >= 2:
            feature_df.loc[index, "last_1_spell_score"] = single_student_df.iloc[1]["spell_score"]
            feature_df.loc[index, "last_1_fa_score"] = single_student_df.iloc[1]["fa_score"]
            feature_df.loc[index, "last_1_game_score"] = single_student_df.iloc[1]["game_score"]
        if single_student_df.shape[0] >= 3:
            feature_df.loc[index, "last_2_spell_score"] = single_student_df.iloc[2]["spell_score"]
            feature_df.loc[index, "last_2_fa_score"] = single_student_df.iloc[2]["fa_score"]
            feature_df.loc[index, "last_2_game_score"] = single_student_df.iloc[2]["game_score"]
        if single_student_df.shape[0] >= 4:
            feature_df.loc[index, "last_3_spell_score"] = single_student_df.iloc[3]["spell_score"]
            feature_df.loc[index, "last_3_fa_score"] = single_student_df.iloc[3]["fa_score"]
            feature_df.loc[index, "last_3_game_score"] = single_student_df.iloc[3]["game_score"]
print(feature_df.shape)
print(feature_df.columns)
print(feature_df.head())

100%|██████████| 204/204 [00:17<00:00, 11.63it/s]

(492, 16)
Index(['student_id', 'lesson_id', 'city_id', 'ts', 'spell_score', 'fa_score',
       'game_score', 'last_1_spell_score', 'last_1_fa_score',
       'last_1_game_score', 'last_2_spell_score', 'last_2_fa_score',
       'last_2_game_score', 'last_3_spell_score', 'last_3_fa_score',
       'last_3_game_score'],
      dtype='object')
   student_id lesson_id city_id          ts  spell_score   fa_score  \
0        5611         1      一线  1755588156    25.666667  15.900000   
1        5611         2    None  1755015533    30.400000  18.263158   
2        5611         3    None  1754563774    53.600000  26.750000   
3        5611        11    None  1756284633     0.000000   0.000000   
4        5611        34    None  1755162289    16.166667   0.000000   

   game_score  last_1_spell_score  last_1_fa_score  last_1_game_score  \
0   -9.463047                 NaN              NaN                NaN   
1   18.095238           25.666667        15.900000          -9.463047   
2   46.666667  

In [33]:
user_df = pd.read_csv(os.path.join(dir_path, "user_profiles.csv"))
print(user_df.shape)
print(user_df.columns)
print(user_df.head())
merged_df = feature_df.merge(
    user_df, 
    on="student_id", 
    how="left"
)
print(merged_df.shape)
print(merged_df.columns)
print(merged_df.head())

(43048, 8)
Index(['student_id', 'mobile', 'nickname', 'sex', 'birthday',
       'cat_listening_level', 'cat_reading_level', 'cat_oral_level'],
      dtype='object')
   student_id       mobile     nickname   sex    birthday  \
0           6  18332532347       内部技术支持   boy  2017-10-09   
1           7  15850775918  158****5918  girl         NaN   
2           8  13520650986  135****0986   boy  2011-07-09   
3           9  17665495215  176****5215   boy  2025-03-03   
4          10  15901557296  159****7296  girl  2020-05-23   

   cat_listening_level  cat_reading_level  cat_oral_level  
0                  1.0                1.0             1.0  
1                  NaN                NaN             NaN  
2                  NaN                NaN             NaN  
3                  NaN                NaN             NaN  
4                  1.0                1.0             1.0  
(492, 23)
Index(['student_id', 'lesson_id', 'city_id', 'ts', 'spell_score', 'fa_score',
       'game_score',

In [34]:
merged_df.to_csv("feature.csv")
merged_df.to_parquet("feature.parquet")

In [79]:
spell_coeff = 0.5
fa_coeff = 0.25
game_coeff = 0.25
value_df = feature_df[["student_id", "lesson_id", "ts"]].copy()
value_df["value"] = (
    feature_df["spell_score"] * spell_coeff + feature_df["fa_score"] * fa_coeff + feature_df["game_score"] * game_coeff + np.random.randn(len(feature_df))
)
print(value_df.shape)
print(value_df.head())

value_df.to_parquet("value.parquet")

(136, 4)
  student_id lesson_id          ts      value
0       5611         3  1754563774   0.101223
1      44395         9  1754564021  42.203237
2      44395        10  1754908594  47.944461
3      42476         2  1754564107  13.562666
4         20        37  1754564208  -0.150542


In [15]:
def parse_input(sent):
    letter_comb_count = 0
    sound_count = 0
    sent_info = {"name": [], "sound": [], "word": [], "other": []}
    for e in re.split(r"[,. ]", sent):
        if e.strip() == "":
            continue
        if e.lower() in ["hello", "says", "hi"]:
            sent_info["other"].append(e)
            continue
        if (e.lower().isalpha() and len(e) == 1) or e.lower() == "space":
            sent_info["name"].append(e)
            if letter_comb_count == 0:
                letter_comb_count = 1
            if e in sent_info["name"]:
                letter_comb_count = 2
            continue
        if e.startswith("/") and e.endswith("/"):
            sound_count += 1
            if e not in sent_info["sound"]:
                sent_info["sound"].append(e)
            continue
        if e not in sent_info["word"]:
            sent_info["word"].append(e)
    if letter_comb_count == 2:
        if len(sent_info["name"]) % 2 == 0:
            nc = {}
            for n in sent_info["name"]:
                if n not in nc:
                    nc[n] = 0
                nc[n] += 1
            if min(nc.values()) == 2:
                sent_info["name"] = sent_info["name"][: len(sent_info["name"]) // 2]
            else:
                letter_comb_count = 1
        else:
            letter_comb_count = 1
    sent_info["letter_comb_count"] = letter_comb_count
    sent_info["sound_count"] = sound_count
    return sent_info

In [17]:
task_info = parse_input("N says /n/, /n/ /n/ newt.")

In [18]:
task_letter = " ".join(task_info["name"])
task_word = task_info["word"][0]
correct_letter_answer = task_letter
correct_word_answer = task_word

In [19]:
print(task_letter)
print(task_word)
print(correct_letter_answer)
print(correct_word_answer)

N
newt
N
newt


In [ ]:
dir_path = "/root/group-shared/jrc/data"
df = pd.read_parquet(os.path.join(dir_path, "feature.parquet"))
print(df.columns)
print(df.shape)
print(df.head())

Index(['student_id', 'lesson_id', 'ts', 'spell_score', 'fa_score',
       'game_score', 'last_1_spell_score', 'last_1_fa_score',
       'last_1_game_score', 'last_2_spell_score', 'last_2_fa_score',
       'last_2_game_score', 'last_3_spell_score', 'last_3_fa_score',
       'last_3_game_score', 'mobile', 'nickname', 'sex', 'birthday',
       'cat_listening_level', 'cat_reading_level', 'cat_oral_level'],
      dtype='object')
(159, 22)
   student_id lesson_id          ts  spell_score   fa_score  game_score  \
0        5611         2  1755015533    30.400000  18.263158   18.095238   
1        5611         3  1754563774     2.750000   0.000000    0.000000   
2       44395         9  1754564021    70.000000  25.613333    0.000000   
3       44395        10  1754908594    60.000000  26.052941   40.000000   
4       44395        11  1755080900    50.333333  27.614286   33.939394   

   last_1_spell_score  last_1_fa_score  last_1_game_score  last_2_spell_score  \
0                 NaN         

: 